In [27]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

tfidf_word_value = []
X = []

readIn = pd.read_csv("../Clean/lemma_result.csv")

X_temp = readIn.iloc[:500, :-1].values
Y = readIn.iloc[:500, -1].values

for sentence in X_temp:
    X.append(sentence[0])

#split data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)
print("Train set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(X_train),(len(X_train[Y_train == 0]) / (len(X_train)*1.))*100,(len(X_train[Y_train == 1]) / (len(X_train)*1.))*100))
print("Test set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(X_test),(len(X_test[Y_test == 0]) / (len(X_test)*1.))*100,(len(X_test[Y_test == 1]) / (len(X_test)*1.))*100))

# TF-IDF
vectorizer = TfidfVectorizer()
df_train = vectorizer.fit_transform(X_train).toarray()
df_test = vectorizer.transform(X_test).toarray()
print(df_train)

TypeError: only integer scalar arrays can be converted to a scalar index

In [7]:
print(len(vectorizer.vocabulary_))

477


In [21]:
# Build the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
embedding_vector_length = 32
vocab_size = len(vectorizer.vocabulary_)
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,     
                                     input_length=vocab_size) )
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(32, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 9218, 32)          294976    
_________________________________________________________________
spatial_dropout1d_10 (Spatia (None, 9218, 32)          0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 50)                16600     
_________________________________________________________________
dropout_13 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 32)                1632      
_________________________________________________________________
dropout_14 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 8)               

In [22]:
history = model.fit(df_train,Y_train,
                  validation_split=0.2, epochs=50, batch_size=64)

Train on 320 samples, validate on 80 samples
Epoch 1/50
320/320 [==============================] - 114s 355ms/sample - loss: 0.7120 - accuracy: 0.5219 - val_loss: 0.6895 - val_accuracy: 0.5500
Epoch 2/50
320/320 [==============================] - 111s 346ms/sample - loss: 0.6980 - accuracy: 0.5531 - val_loss: 0.6900 - val_accuracy: 0.5500
Epoch 3/50
320/320 [==============================] - 104s 324ms/sample - loss: 0.6873 - accuracy: 0.5500 - val_loss: 0.6905 - val_accuracy: 0.5500
Epoch 4/50
320/320 [==============================] - 105s 329ms/sample - loss: 0.7412 - accuracy: 0.4344 - val_loss: 0.6911 - val_accuracy: 0.5500
Epoch 5/50
320/320 [==============================] - 145s 453ms/sample - loss: 0.7232 - accuracy: 0.4938 - val_loss: 0.6911 - val_accuracy: 0.5500
Epoch 6/50
320/320 [==============================] - 197s 615ms/sample - loss: 0.6927 - accuracy: 0.5594 - val_loss: 0.6911 - val_accuracy: 0.5500
Epoch 7/50
320/320 [==============================] - 261s 815ms/sa

KeyboardInterrupt: 

In [61]:
from tensorflow.keras.models import model_from_yaml

# serialize model to YAML
model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
